<a href="https://colab.research.google.com/github/9M3a1h3d9i9/NeuroResilience-DRL/blob/main/NeuroResilience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. نصب کتابخانه های مورد نیاز (این خط را در کولب اجرا کنید)


In [ ]:
!pip install gymnasium stable-baselines3 networkx matplotlib shimmy


In [2]:
import gymnasium as gym
from gymnasium import spaces
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


#  بخش اول: تعریف محیط گراف (Custom Environment)


In [7]:
class GraphInterdictionEnv(gym.Env):
    """
    محیط سفارشی برای مسئله کاهش جریان شبکه (Interdiction).
    هدف عامل: حذف یال‌هایی که بیشترین تاثیر را در کاهش Max-Flow دارند.
    """
    def __init__(self, num_nodes=20, num_edges_to_remove=3):
        super(GraphInterdictionEnv, self).__init__()
        self.num_nodes = num_nodes
        self.budget = num_edges_to_remove

        # ساخت گراف اولیه (مدل Barabasi-Albert برای شبیه سازی شبکه های واقعی)
        self.base_graph = nx.barabasi_albert_graph(num_nodes, 3)
        # اختصاص ظرفیت به یال ها (فعلا همه 1)
        for u, v in self.base_graph.edges():
            self.base_graph[u][v]['capacity'] = 1.0

        self.source = 0
        self.target = num_nodes - 1

        # تعریف فضای کنش (Action Space): انتخاب یکی از یال ها برای حذف
        # تعداد یال ها ممکن است تغییر کند، پس حد بالا را در نظر می گیریم
        self.max_edges = self.num_nodes * (self.num_nodes - 1) // 2
        self.action_space = spaces.Discrete(self.max_edges)

        # تعریف فضای حالت (Observation Space): ماتریس مجاورت گراف
        self.observation_space = spaces.Box(low=0, high=1, shape=(num_nodes, num_nodes), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # بازیابی گراف اولیه
        self.current_graph = self.base_graph.copy()
        self.steps_taken = 0

        # لیست یال ها برای نگاشت Action به Edge
        self.edge_list = list(self.current_graph.edges())

        obs = nx.to_numpy_array(self.current_graph).astype(np.float32) # Cast to float32
        return obs, {}

    def step(self, action):
        # بررسی معتبر بودن کنش
        if action < len(self.edge_list):
            edge_to_remove = self.edge_list[action]

            if self.current_graph.has_edge(*edge_to_remove):
                self.current_graph.remove_edge(*edge_to_remove)

        # محاسبه پاداش: هرچقدر Max-Flow کمتر شود، پاداش بیشتر است
        # ما می خواهیم عامل یاد بگیرد جریان را "قطع" کند.
        try:
            flow_value = nx.maximum_flow_value(self.current_graph, self.source, self.target)
        except:
            flow_value = 0 # اگر مسیر قطع شده باشد

        # پاداش = منفی جریان (چون RL می خواهد پاداش را زیاد کند، پس جریان را کم می کند)
        reward = -flow_value

        self.steps_taken += 1
        terminated = self.steps_taken >= self.budget
        truncated = False

        obs = nx.to_numpy_array(self.current_graph).astype(np.float32) # Cast to float32
        return obs, reward, terminated, truncated, {}

# بخش دوم: آموزش مدل (Training)

## ساخت محیط


In [8]:
env = GraphInterdictionEnv(num_nodes=30, num_edges_to_remove=5)

# چک کردن استاندارد بودن محیط
check_env(env)

# تعریف مدل PPO (عامل هوشمند)
model = PPO("MlpPolicy", env, verbose=1, learning_rate=0.001)

print("شروع آموزش عامل هوشمند (این مرحله چند دقیقه طول می کشد)...")
# آموزش به اندازه 20,000 گام
model.learn(total_timesteps=20000)

# --- بخش سوم: تست و ارزیابی ---
print("\nآموزش تمام شد. حالا بیایید تست کنیم!")

obs, _ = env.reset()
total_reward = 0
print(f"تعداد گره ها: {env.num_nodes}, هدف: حذف 5 یال حیاتی")

for i in range(5):
    action, _states = model.predict(obs)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward
    print(f"گام {i+1}: عامل یال شماره {action} را حذف کرد. پاداش لحظه ای: {reward}")

print(f"\nعملکرد نهایی عامل: {total_reward} (هرچه به 0 نزدیکتر باشد یعنی جریان را بهتر قطع کرده)")

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/env_checker.py:316: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
شروع آموزش عامل هوشمند (این مرحله چند دقیقه طول می کشد)...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5        |
|    ep_rew_mean     | -14.8    |
| time/              |          |
|    fps             | 311      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | -14.9       |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.029722467 |
|    clip_fraction        | 0.46        |
|    clip_range           | 0.2         |
|    entropy_loss         | -6.05       |
|    explained_variance   | 9.55e-05    |
|    learning_rate        | 0.

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | -14.9       |
| time/                   |             |
|    fps                  | 260         |
|    iterations           | 3           |
|    time_elapsed         | 23          |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.027576786 |
|    clip_fraction        | 0.423       |
|    clip_range           | 0.2         |
|    entropy_loss         | -6.02       |
|    explained_variance   | 0.000478    |
|    learning_rate        | 0.001       |
|    loss                 | 9.51        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.0378     |
|    value_loss           | 17.5        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5     

#  بخش چهارم: رسم نمودار (اختیاری - نیاز به Callback دارد اما اینجا ساده تست کردیم)


# برای رسم دقیق نمودار Learning Curve در اجراهای بعدی از Monitor استفاده می کنیم.